In [14]:
import numpy as np
import math, os, glob
import netCDF4 as cdf
import xarray as xr
from datetime import datetime, timedelta, timezone

import datetime as dt
import pandas as pd

In [15]:
domains        = 1                                                                # check this always
w              = '../../../DATA/WRF/wrfinput/'   # check this always
w_geo          = '../../../DATA/WRF/domains/4_domains/'# check this always
pre            = 'wrfinput_d0'
pre2           = 'geo_em.d0'
sim_time       = '2022-08-01 00:00:00','2022-08-31 23:00:00' 
dates          = pd.to_datetime(sim_time[0]).strftime('%Y-%m-%d')
pos            = '.nc'  
projection_wrf = 'Lambert Conformal'

In [16]:
wet_out          = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'
cpool_out        = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'
t_ann_out        = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'

In [17]:
dom              = domains 
wrf_inp          = w + pre  + str(dom)
wrf_geo_p        = w_geo + pre2 + str(dom) + pos

## join CH4 and CO2 biogenic dataset

######### READ CH4 BIO -- Kaplan inventory
cpoolp  = cpool_out +'CPOOL_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)
tannp   = t_ann_out + 'T_ANN_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)
wetmapp = wet_out + 'WETMAP_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)

######### READ CO2 BIO ---- prec vprm
vprm_inp  = '../../../DATA/sources/VPRM/co2_bio_input/regrid/'
evi_minp  = vprm_inp + 'd0%s/VPRM_input_EVI_MIN_%s.nc'%(dom,pd.to_datetime(dates).year)
evi_maxp  = vprm_inp + 'd0%s/VPRM_input_EVI_MAX_%s.nc'%(dom,pd.to_datetime(dates).year)
evip      = vprm_inp + 'd0%s/VPRM_input_EVI_%s.nc'%(dom,pd.to_datetime(dates).year)
lswi_minp = vprm_inp + 'd0%s/VPRM_input_LSWI_MIN_%s.nc'%(dom,pd.to_datetime(dates).year)
lswi_maxp = vprm_inp + 'd0%s/VPRM_input_LSWI_MAX_%s.nc'%(dom,pd.to_datetime(dates).year)
lswip     = vprm_inp + 'd0%s/VPRM_input_LSWI_%s.nc'%(dom,pd.to_datetime(dates).year)
vegfrap   = vprm_inp + 'd0%s/VPRM_input_VEG_FRA_%s.nc'%(dom,pd.to_datetime(dates).year)
    
file_out = '../../../DATA/sources/VPRM/'

In [18]:
######### READ CH4 BIO -- Kaplan inventory
cpool  = xr.open_dataset(cpoolp)
tann   = xr.open_dataset(tannp)
wetmap = xr.open_dataset(wetmapp)

######### READ CO2 BIO ---- prec vprm
## 1) Interpolate EVI for all days -- data each 8 eight (by measured satelite)
evi = xr.open_dataset(evip)
lswi = xr.open_dataset(lswip)

## 2) This data is unique for all
evi_min = xr.open_dataset(evi_minp)
evi_max = xr.open_dataset(evi_maxp)
lswi_min = xr.open_dataset(lswi_minp)
lswi_max = xr.open_dataset(lswi_maxp)
vegfra = xr.open_dataset(vegfrap)

#wrf_i       = cdf.Dataset(wrf_inp)
wrf_geo     = cdf.Dataset(wrf_inp)

In [35]:
all_time = pd.date_range("2022-01-01", "2022-12-31",freq="D").strftime('%Y-%m-%d')

In [34]:
#### drop the EVI and LSWI data for us time:
star = list(all_time).index(sim_time[0][0:10])
end = list(all_time).index(sim_time[1][0:10])
evi_month = evi.sel(time=slice(star,end))
lswi_month = lswi.sel(time=slice(star,end))

In [38]:
evi_month.variables['evi'].values.shape

(8, 31, 124, 204)

In [ ]:
print('generating netcdf file..')
wrf_input     = wrf_geo
dim_emi_items = list(wrf_input.dimensions.items())
domain        = dom
projection    = projection_wrf

bytime        = pd.date_range("2022-08-01", freq="D", periods=31)
for m in bytime:
    date          = pd.to_datetime(m).strftime('%Y-%m-%d')+'_00:00:00'
    filename      = file_out + 'vprm_input_d0%s_%s'%(dom,str(date).replace(":","_"))
    dataset       = cdf.Dataset(filename,'w',format='NETCDF4_CLASSIC')

    # Dimensions
    len           = dataset.createDimension('DateStrLen',19)
    time          = dataset.createDimension('Time', 1)
    lon           = dataset.createDimension(dim_emi_items[2][0], wrf_input.getncattr('WEST-EAST_PATCH_END_UNSTAG'))
    lat           = dataset.createDimension(dim_emi_items[3][0], wrf_input.getncattr('SOUTH-NORTH_PATCH_END_UNSTAG'))
    vegfran       = dataset.createDimension('vprm_vgcls',8)
    levels        = dataset.createDimension('zdim',1)

    # Variables
    time          = dataset.createVariable('Times', 'S1',('Time','DateStrLen'))
    lat           = dataset.createVariable('XLAT', 'f4', (dim_emi_items[3][0],dim_emi_items[2][0]))
    lon           = dataset.createVariable('XLONG', 'f4', (dim_emi_items[3][0],dim_emi_items[2][0]))

    evi_min_in    = dataset.createVariable('EVI_MIN', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    evi_max_in    = dataset.createVariable('EVI_MAX', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    evi_in        = dataset.createVariable('EVI', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    lswi_min_in   = dataset.createVariable('LSWI_MIN', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    lswi_max_in   = dataset.createVariable('LSWI_MAX', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    lswi_in       = dataset.createVariable('LSWI', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    vegfra_in     = dataset.createVariable('VEGFRA_VPRM', 'f4', ('Time','vprm_vgcls',dim_emi_items[3][0],dim_emi_items[2][0]))
    cpool_in      = dataset.createVariable('CPOOL', 'f4', ('Time','zdim',dim_emi_items[3][0],dim_emi_items[2][0]))
    wetmap_in     = dataset.createVariable('WETMAP', 'f4', ('Time','zdim',dim_emi_items[3][0],dim_emi_items[2][0]))
    tann_in       = dataset.createVariable('T_ANN', 'f4', ('Time','zdim',dim_emi_items[3][0],dim_emi_items[2][0]))

    # data
    time[:,:]           = np.array([" ".join(date).split()], dtype = 'S19')
    evi_min_in[:,:,:]   = evi_min.variables['evi_min'].values
    evi_max_in[:,:,:]   = evi_max.variables['evi_max'].values
    evi_in[:,:,:]       = evi_month.variables['evi'].values
    lswi_min_in[:,:,:]  = lswi_min.variables['lswi_min'].values
    lswi_max_in[:,:,:]  = lswi_max.variables['lswi_max'].values
    lswi_in[:,:,:]      = lswi_month.variables['lswi'].values
    vegfra_in[:,:,:]    = vegfra.variables['vegetation_fraction_map'].values
    cpool_in[:,:]       = cpool.variables['CPOOL'].values
    wetmap_in[:,:]      = wetmap.variables['WETMAP'].values
    tann_in[:,:]        = tann.variables['T_ANN'].values

    lat[:,:]            = np.array(wrf_input.variables['XLAT'])
    lon[:,:]            = np.array(wrf_input.variables['XLONG'])


    # Attributes global
    lista = ['CEN_LAT','CEN_LON','TRUELAT1','TRUELAT2','MOAD_CEN_LAT','STAND_LON','POLE_LAT','POLE_LON','MAP_PROJ',
            'NUM_LAND_CAT','ISWATER','ISLAKE','ISICE','ISURBAN','ISOILWATER','MMINLU','GMT','JULYR','JULDAY']
    import time
    dataset.Source    = 'WRF input file created by Benavente R.N., (2023). Reference: vprm_shapeshifter, v1.5 (MPI-BGC Jena 2019)'
    dataset.History   = 'Created ' + time.ctime(time.time())

    for i,j in enumerate(lista):
        dataset.setncattr(j,wrf_input.getncattr(j))

    # Attributes variables
    evi_min_in.long_name    = 'minimum annual EVI index value'
    evi_min_in.FieldType    = 104
    evi_min_in.MemoryOrder  = 'XYZ'
    evi_min_in.units        = ''
    evi_min_in.description  = 'Minimal value of EVI index'
    evi_min_in.stagger      = 'M'
    evi_min_in.coordinates  = 'XLONG XLAT'

    evi_max_in.long_name    = 'maximum annual EVI index value'
    evi_max_in.FieldType    = 104
    evi_max_in.MemoryOrder  = 'XYZ'
    evi_max_in.units        = ''
    evi_max_in.description  = 'Maximal value of EVI index'
    evi_max_in.stagger      = 'M'
    evi_max_in.coordinates  = 'XLONG XLAT'

    evi_in.long_name        = 'EVI index value'
    evi_in.FieldType        = 104
    evi_in.MemoryOrder      = 'XYZ'
    evi_in.units            = ''
    evi_in.description      = 'Value of EVI index'
    evi_in.stagger          = 'M'
    evi_in.coordinates      = 'XLONG XLAT'

    lswi_min_in.long_name   = 'minimum annual LSWI index value'
    lswi_min_in.FieldType   = 104
    lswi_min_in.MemoryOrder = 'XYZ'
    lswi_min_in.units       = ''
    lswi_min_in.description = 'Minimal value of lswi index'
    lswi_min_in.stagger     = 'M'
    lswi_min_in.coordinates = 'XLONG XLAT'

    lswi_max_in.long_name   = 'maximum annual LSWI index value'
    lswi_max_in.FieldType   = 104
    lswi_max_in.MemoryOrder = 'XYZ'
    lswi_max_in.units       = ''
    lswi_max_in.description = 'Maximal value of lswi index'
    lswi_max_in.stagger     = 'M'
    lswi_max_in.coordinates = 'XLONG XLAT'

    lswi_in.long_name       = 'LSWI index value'
    lswi_in.FieldType       = 104
    lswi_in.MemoryOrder     = 'XYZ'
    lswi_in.units           = ''
    lswi_in.description     = 'Value of lswi index'
    lswi_in.stagger         = 'M'
    lswi_in.coordinates     = 'XLONG XLAT'

    vegfra_in.long_name    = 'VPRM vegetation fraction'
    vegfra_in.FieldType    = 104
    vegfra_in.MemoryOrder  = 'XYZ'
    vegfra_in.units        = ''
    vegfra_in.description  = 'Vegetation fraction for VPRM'
    vegfra_in.stagger      = 'M'
    vegfra_in.coordinates  = 'XLONG XLAT'

    cpool_in.long_name    = 'LPJ Carbon pool'
    cpool_in.FieldType    = 104
    cpool_in.MemoryOrder  = 'XYZ'
    cpool_in.units        = '$kgC/m^{2}$'
    cpool_in.description  = 'Carbon pool value for Kaplan model'
    cpool_in.stagger      = 'M'
    cpool_in.coordinates  = 'XLONG XLAT'

    wetmap_in.long_name   = 'Kaplan potential wetland map'
    wetmap_in.FieldType   = 104
    wetmap_in.MemoryOrder = 'XYZ'
    wetmap_in.units       = '1 Woolong'
    wetmap_in.description = 'Wetland map for Kaplan model'
    wetmap_in.stagger     = 'M'
    wetmap_in.coordinates = 'XLONG XLAT'

    tann_in.long_name     = 'mean annual temperature'
    tann_in.FieldType     = 104
    tann_in.MemoryOrder   = 'XYZ'
    tann_in.units         = 'K'
    tann_in.description   = 'Annual mean temperature for vegetation classes'
    tann_in.stagger       = 'M'
    tann_in.coordinates   = 'XLONG XLAT'

    dataset.close()

print('DONE GENERATING THE VPRM INPUT')